<a href="https://colab.research.google.com/github/ridhimagarg/PyTorchBook/blob/main/Chapter02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install torchviz

  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=6c8882199598e2b91fa72c79d3230094e11c3e82c61c443ab827a43d77d12764
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz


In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

## Changing the training loop

In [3]:
def make_train_step(model, loss_fn, optimizer):

  def perform_train_step(x, y):

    model.train()
    yhat = model(x)

    loss = loss_fn(yhat, y)

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    return loss.item()

  return perform_train_step

In [4]:
## Changing the model configuration code

device = "cuda" if torch.cuda.is_available() else "cpu"

lr = 0.1

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1,1)).to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)

loss_fn = nn.MSELoss(reduction="mean")

train_step = make_train_step(model, loss_fn, optimizer)

In [5]:
train_step

<function __main__.make_train_step.<locals>.perform_train_step>

In [6]:
true_b = 1
true_w = 2
N = 100
# Data Generation
np.random.seed(42)
x = np.random.rand(N, 1)
epsilon = (.1 * np.random.randn(N, 1))
y = true_b + true_w * x + epsilon

idx = np.arange(N)
np.random.shuffle(idx)
# Uses first 80 random indices for train
train_idx = idx[:int(N*.8)]
# Uses the remaining indices for validation
val_idx = idx[int(N*.8):]
# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

In [7]:
## Changing the model training code

n_epochs = 1000

losses = []

for epoch in range(n_epochs):

  loss = train_step(x_train_tensor, y_train_tensor)

  losses.append(loss)



In [8]:
# Checks model's parameters
print(model.state_dict())


OrderedDict([('0.weight', tensor([[1.9690]])), ('0.bias', tensor([1.0235]))])


## Dataset

In [9]:
class CustomDataset(Dataset):

  def __init__(self, x_tensor, y_tensor):
    self.x = x_tensor
    self.y = y_tensor

  def __getitem__(self, index):
    return (self.x[index], self.y[index])

  def __len__(self):
      return len(self.x)

In [10]:
x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


In [11]:
print(train_data[1])

(tensor([0.0636]), tensor([1.1928]))


## Tensor Dataset for more simplicity instead of custom.

In [12]:
train_data = TensorDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


## Now using DataLoader for mini na

In [15]:
# %%writefile data_preparation/v1.py

## Here we are not creating GPU tensors, it is expensive.
## Will convert it later.
x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = TensorDataset(x_train_tensor, y_train_tensor)

train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

lr = 0.1

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1,1)).to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)

loss_fn = nn.MSELoss(reduction="mean")

train_step = make_train_step(model, loss_fn, optimizer)

In [17]:

n_epochs = 1000

losses = []

for epoch in range(n_epochs):

  mini_batch_losses = []

  for x_batch, y_batch in train_loader:

    ## Here changing the CPU tensors to GPU
    ## we sent only one mini-batch to the device, as opposed to sending the whole training set
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    mini_batch_loss = train_step(x_batch, y_batch)
    mini_batch_losses.append(mini_batch_loss)

  loss = np.mean(mini_batch_losses)

  losses.append(loss)


In [18]:
def mini_batch(device, data_loader, step):

  mini_batch_losses = []

  for x_batch, y_batch in train_loader:

    ## Here changing the CPU tensors to GPU
    ## we sent only one mini-batch to the device, as opposed to sending the whole training set
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    mini_batch_loss = train_step(x_batch, y_batch)
    mini_batch_losses.append(mini_batch_loss)

  loss = np.mean(mini_batch_losses)

  return loss


In [19]:
## We are reducing the no. of epochs now, becauase doing extra steps inside mini batch loop
n_epochs = 200

losses = []

for epoch in range(n_epochs):

  loss = mini_batch(device, train_loader, train_step) 

  losses.append(loss)